In [1]:
import numpy as np
import sdv
import scipy
import pandas as pd
import sdmetrics
from sdv.metadata import SingleTableMetadata
from sdmetrics.reports import utils
from sdv.single_table import TVAESynthesizer
from sdv.single_table import CTGANSynthesizer
from sdv.evaluation.single_table import evaluate_quality

import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer

import hyperimpute
from hyperimpute.plugins.imputers import Imputers, ImputerPlugin
imputers = Imputers()

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from scipy import stats

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from keras import metrics

import tensorflow

from tensorflow.keras.models import Model

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


2024-11-05 11:16:01.103996: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 11:16:01.132733: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-05 11:16:01.998723: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
kihu_aineisto = pd.read_csv("../kihu_data_cleaned.csv", index_col=0)

# Drop derived variables
kihu_aineisto.drop(["BMI 1", "BMI 2", "BMI 3", "BMI 4", "Testo/Cort (nmol/L) 1", "Testo/Cort (nmol/L) 2",
                    "V02max, ml/kg/min 1", "V02max, ml/kg/min 2", "V02max, ml/kg/min 3", "V02max, ml/kg/min 4",
                    "Testo/Cort (nmol/L) 3", "Testo/Cort (nmol/L) 4", "Kesto (days) Koko jakso",
                    "Harjoittelua AVG (h) /viikko Koko jakso",
                   "Harjoittelua krt/vko Koko jakso", "Jukm/viikko Koko jakso", "PK% Koko jakso",
                   "VK% Koko jakso", "MK% Koko jakso"], axis=1, inplace=True)

In [3]:
# Rounding values taken as the minimum number of decimals on the original dataset

rounding_scheme ={"Voimaryhmä":0,"Age":0, "Training years":0, "TrainTimesWkprev2months":0, "Runningkmvkoprev2months":0,
                  "Weight 1":1, "Weight 2":1,"Weight 3":1,"Weight 4":1, "Fat % 1":1, "Fat % 2":1, "Fat % 3":1, "Fat % 4":1,
                  "Korsol (nmol/l) 1":1, "Korsol (nmol/l) 2":1, "Korsol (nmol/l) 3":1, "Korsol (nmol/l) 4":1,
                  "Testo (nmol/l) 1":1, "Testo (nmol/l) 2":1, "Testo (nmol/l) 3":1, "Testo (nmol/l) 4":1,
                  "1 RM (kg) 1":1, "1 RM (kg) 2":1, "1 RM (kg) 3":1, "1 RM (kg) 4":1,
                  "CMJ (cm) 1":1, "CMJ (cm) 2":1, "CMJ (cm) 3":1, "CMJ (cm) 4":1, "Time3000m 1":1, "Time3000m 2":1, "Time3000m 3":1,
                  "VO2max (l/min) 1":1, "VO2max (l/min) 2":1, "VO2max (l/min) 3":1, "VO2max (l/min) 4":1,
                  "Vmax  (km/h) 1": 1, "Vmax  (km/h) 2": 1, "Vmax  (km/h) 3": 1, "Vmax  (km/h) 4": 1, 
                  "HR max (bpm) 1":0, "HR max (bpm) 2":0, "HR max (bpm) 3":0, "HR max (bpm) 4":0, "Lamax (mmol/l) 1": 1,
                  "Lamax (mmol/l) 2": 1, "Lamax (mmol/l) 3": 1, "Lamax (mmol/l) 4": 1, "VAnT (km/h) 1":1, "VAnT (km/h) 2":1, 
                  "VAnT (km/h) 3":1, "VAnT (km/h) 4":1,  "VAerT (km/h) 1":1, "VAerT (km/h) 2":1, "VAerT (km/h) 3":1, "VAerT (km/h) 4":1,
                  "MART Vmax (m/s) 1":1, "MART Vmax (m/s) 2":1, "MART Vmax (m/s) 3":1, "MART Vmax (m/s) 4":1,
                  "MART LAmax (mM) 1":1, "MART LAmax (mM) 2":1, "MART LAmax (mM) 3":1,
                  "MART LAmax (mM) 4":1, "R economy (ml/kg/km) 1":1, "R economy (ml/kg/km) 2":1, "R economy (ml/kg/km) 3":1,
                  "R economy (ml/kg/km) 4":1, "Night HR1":1, "Night HR2":1, "Night HR3":1, "Night HR4":1, "SDRR1":0, "SDRR2":0,
                  "SDRR3":0, "SDRR4":0, "RMSSD1":0, "RMSSD2":0, "RMSSD3":0, "RMSSD4":0, "Abs.Relaxation1":1,
                  "Abs.Relaxation2":1, "Abs.Relaxation3":1, "Abs.Relaxation4":1, "Abs.Stress1": 1, "Abs.Stress2": 1,
                  "Abs.Stress3": 1, "Abs.Stress4": 1, "VLF1":2, "VLF2":2, "VLF3":2, "VLF4":2, "LF1":2, "LF2":2, "LF3":2, "LF4":2,
                  "HF1":2, "HF2":2, "HF3":2, "HF4":2, "HF2 1":2, "HF2 2":2, "HF2 3":2, "HF2 4":2, "TP1":2, "TP2":2, "TP3":2, "TP4":2,
                  "Kesto (days) 1jakso":0, "Kesto (days) 2jakso":0, "Kesto (days) 3jakso":0, "Kesto (days) Koko jakso":0,
                  "Harjoittelua AVG (h) /viikko 1jakso":1, "Harjoittelua AVG (h) /viikko2jakso":1, "Harjoittelua AVG (h) /viikko 3jakso":1,
                  "Harjoittelua AVG (h) /viikko Koko jakso":1, "Harjoittelua krt/vko 1jakso":1, "Harjoittelua krt/vko 2jakso":1,
                  "Harjoittelua krt/vko 3jakso":1, "Harjoittelua krt/vko Koko jakso":1, "Jukm/viikko 1jakso":1,
                  "Jukm/viikko 2jakso":1, "Jukm/viikko 3jakso":1, "Jukm/viikko Koko jakso":1, "PK%  1jakso":1, "PK% 2jakso":1,
                  "PK% 3jakso":1, "PK% Koko jakso":1,"VK% 1jakso":1, "VK% 2jakso":1,
                  "VK% 3jakso":1, "VK% Koko jakso":1, "MK% 1jakso":1, "MK% 2jakso":1,
                  "MK% 3jakso":1, "MK% Koko jakso":1, "Tehoharj krt": 0}



In [4]:
def rm_half_nan(df, df_with_nans):
    mask = np.sum(df_with_nans.isna(),axis=1)<df_with_nans.shape[1]/2
    return df[mask]

In [5]:
# Hyperimpute is an automatic imputation tool
# Preserves links between variables better than KNN

# imputer_hyper1 = imputers.get("hyperimpute")
# kihu_aineisto_hyper = imputer_hyper1.fit_transform(kihu_aineisto)

# kihu_aineisto_hyper = pd.DataFrame(kihu_aineisto_hyper.to_numpy(), columns=kihu_aineisto.columns.values)

In [6]:
kihu_aineisto.drop(['Time3000m 1','Time3000m 2','Time3000m 3'], axis=1, inplace=True)
# kihu_aineisto_hyper.drop(['Time3000m 1','Time3000m 2','Time3000m 3'], axis=1, inplace=True)

In [7]:
# kihu_aineisto_hyper.reset_index(inplace=True, drop=True)
kihu_aineisto.reset_index(inplace=True, drop=True)

In [8]:
# Divide column names by their measurement points
def create_profiles_and_tprograms(aineisto):
    first_cols = []
    second_cols = []
    third_cols = []
    fourth_cols = []

    for col_name in aineisto.columns:
        jakso_name = col_name[-6:]
        if col_name[-1] == "2" or jakso_name == "1jakso":
            second_cols.append(col_name)
        elif col_name[-1] == "3" or jakso_name == "2jakso":
            third_cols.append(col_name)
        elif col_name[-1] == "4" or jakso_name == "3jakso" or col_name == 'Tehoharj krt':
            fourth_cols.append(col_name)
        else:
            first_cols.append(col_name)
            
    aineisto_1 = aineisto[first_cols]
    aineisto_2 = aineisto[second_cols]
    aineisto_3 = aineisto[third_cols]
    aineisto_4 = aineisto[fourth_cols]
    
    # Non changing variables into the measurement results
    baseline_columns = ["Gender", "Voimaryhmä","Kestävyysryhmä", "Age", "Training years",
                        "TrainTimesWkprev2months", "Runningkmvkoprev2months", "Height"]

    # Treat the baseline + lab measurement as a "new runner profile"

    henkilot_1 = aineisto_1.copy()

    muuttujat = henkilot_1.columns

    henkilot_2 = pd.concat([aineisto[baseline_columns], aineisto_2.iloc[:,0:-7 ]], axis=1).set_axis(muuttujat,axis=1)
    henkilot_3 = pd.concat([aineisto[baseline_columns], aineisto_3.iloc[:,0:-7 ]], axis=1).set_axis(muuttujat,axis=1)
    henkilot_4 = pd.concat([aineisto[baseline_columns], aineisto_4.iloc[:,0:-8 ]], axis=1).set_axis(muuttujat,axis=1)

    runner_profiles_fold = pd.concat([henkilot_1,henkilot_2,henkilot_3,henkilot_4],axis= 0)

    runner_profiles_fold.reset_index(inplace=True)

    training_program_1 = aineisto_2.iloc[:,-7: ]
    zeros_122 = np.zeros(aineisto_4.shape[0])
    tehoharjkrt = pd.DataFrame(zeros_122, columns=['Tehoharj krt'])

    training_program_1 = pd.concat([training_program_1.reset_index(drop=True), tehoharjkrt],axis=1)
    training_program_2 = aineisto_3.iloc[:,-7: ]
    training_program_3 = aineisto_4.iloc[:,-8: ]

    cols = training_program_1.columns

    tehoharj_training_program_2 = np.zeros(aineisto_4.shape[0])
    tehoharj_training_program_2[62:] = training_program_3.iloc[62:]['Tehoharj krt'].to_numpy()
    tehoharj_training_program_3 = np.zeros(aineisto_4.shape[0])
    tehoharj_training_program_3[0:62] = training_program_3.iloc[0:62]['Tehoharj krt'].to_numpy()

    training_program_2 = pd.concat([training_program_2.reset_index(drop=True),
                                    pd.DataFrame(tehoharj_training_program_2, columns=['Tehoharj krt'])],axis=1).set_axis(cols,axis=1)

    training_program_3 = training_program_3.drop(['Tehoharj krt'],axis=1)

    training_program_3 = pd.concat([training_program_3.reset_index(drop=True),
                                    pd.DataFrame(tehoharj_training_program_3,
                                                 columns=['Tehoharj krt'])],axis=1
                                    ).set_axis(cols,axis=1)

    training_program_2.reset_index(inplace=True,drop=True)
    training_program_3.reset_index(inplace=True,drop=True)

    training_programs = pd.concat([training_program_1,training_program_2,training_program_3],axis= 0)

    training_programs.reset_index(inplace=True)

    return runner_profiles_fold.rename(columns={'index':'ID'}), training_programs.rename(columns={'index':'ID'})

In [9]:
second_col_names = []

for col_name in kihu_aineisto.columns:
    jakso_name = col_name[-6:]
    if col_name[-1] == "2" or jakso_name == "1jakso":
        second_col_names.append(col_name)

baseline_columns = ["Gender", "Voimaryhmä","Kestävyysryhmä", "Age", "Training years",
                        "TrainTimesWkprev2months", "Runningkmvkoprev2months", "Height"]

cols_baseline_measure_2 = np.concatenate([baseline_columns, second_col_names])


In [11]:
n_folds = 10
kf = KFold(n_splits=n_folds, random_state=19, shuffle=True)

# Fold em
for fold, (train_index, test_index) in enumerate(kf.split(kihu_aineisto)):
    kihu_aineisto.iloc[train_index].to_csv(f"./train_sets/kihu_{fold}.csv")
    kihu_aineisto.iloc[test_index].to_csv(f"./test_sets/kihu_{fold}.csv")

In [12]:
kihu_train_list = []
kihu_test_list = []
runner_profiles_list_test = []
train_programs_list_test = []

# Read em
for fold, (train_index, test_index) in enumerate(kf.split(kihu_aineisto)):
    kihu_train_list.append(pd.read_csv(f"train_sets/kihu_{fold}.csv",index_col=0))
    kihu_test_i = pd.read_csv(f"test_sets/kihu_{fold}.csv",index_col=0)
    kihu_test_list.append(kihu_test_i)

In [13]:
runner_profiles_list =[]
train_programs_list= []

for df in kihu_train_list:
    runner_profiles, train_programs = create_profiles_and_tprograms(df)
    runner_profiles_list.append(runner_profiles)
    train_programs_list.append(train_programs)

for test_set in kihu_test_list:
    runner_profiles_test_i, train_programs_test_i = create_profiles_and_tprograms(test_set)
    runner_profiles_list_test.append(runner_profiles_test_i)
    train_programs_list_test.append(train_programs_test_i)



In [14]:
profiles_train_list = []
profiles_test_list = []
nan_cutoff = 100

for fold, (train_index, test_index) in enumerate(kf.split(kihu_aineisto)):
    profile_nans_train, program_nans_train = create_profiles_and_tprograms(kihu_aineisto.iloc[train_index].isna())
    under_50_NA_train = np.sum(profile_nans_train.drop('ID',axis=1),axis=1) < nan_cutoff

    profiles_train_list.append(runner_profiles_list[fold][under_50_NA_train])

    profile_nans_test, program_nans_test = create_profiles_and_tprograms(kihu_aineisto.iloc[test_index].isna())
    under_50_NA_test = np.sum(profile_nans_test.drop('ID',axis=1),axis=1) < nan_cutoff
    profiles_test_list.append(runner_profiles_list_test[fold][under_50_NA_test])

       


In [15]:
## Check nan percentage
for fold, (train_index, test_index) in enumerate(kf.split(kihu_aineisto)):
    kihu_nans, train_nans = create_profiles_and_tprograms(kihu_aineisto.iloc[train_index].isna())
    print(kihu_aineisto.iloc[train_index].isna().sum().sum()/(kihu_aineisto.iloc[train_index].shape[0]*kihu_aineisto.iloc[train_index].shape[1]))
## ~30%

0.30712773465067045
0.3046577275935074
0.3216083916083916
0.31433566433566434
0.3297902097902098
0.3032167832167832
0.32034965034965035
0.30643356643356645
0.3079020979020979
0.3118181818181818


In [15]:
profiles_train_hyper_list = []
profiles_test_hyper_list = []

for fold, profiles in enumerate(profiles_train_list):
    id_column_train = profiles_train_list[fold]['ID']
    id_column_test = profiles_test_list[fold]['ID']

    hyper_imputer = imputers.get("hyperimpute")
    profiles_train_hyper = hyper_imputer.fit_transform(profiles.iloc[:,1:])
    profiles_test_hyper = hyper_imputer.transform(profiles_test_list[fold].iloc[:,1:])
    profiles_train_hyper_list.append(pd.concat([id_column_train,profiles_train_hyper],axis=1))
    profiles_test_hyper_list.append(pd.concat([id_column_test,profiles_test_hyper],axis=1))

/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10

In [59]:
profiles_train_hyper_nonans_list = []
for i, profile in enumerate(profiles_train_hyper_list):
    profile_nonans = rm_half_nan(profile, profiles_train_list[i])
    profiles_train_hyper_nonans_list.append(profile_nonans)
    profile_nonans.to_csv(f"./final_sets/real_train/profiles_{i}.csv")

profiles_test_hyper_nonans_list = []
for i, profile in enumerate(profiles_test_hyper_list):
    profile_nonans = rm_half_nan(profile, profiles_test_list[i])
    profiles_test_hyper_nonans_list.append(profile_nonans)
    profile_nonans.to_csv(f"./final_sets/real_test/profiles_{i}.csv")

In [17]:
# Constraints for TVAE
# Categorical values cant have constraints in sdv

constraint_list = []
numerical_columns = runner_profiles_list[0].drop(['ID','Gender', 'Voimaryhmä', 'Kestävyysryhmä'], axis=1).columns
for col in numerical_columns:
    my_constraint = {
        'constraint_class': 'Positive',
        'constraint_parameters': {
            'column_name': col
        }
    }
    constraint_list.append(my_constraint)
    


In [18]:
def generate_TVAE_hyper_profiles(train_data, n_synth):
    if 'ID' in train_data.columns:
        train_data.drop('ID', axis=1, inplace=True)
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(train_data)
    sdv_TVAE_hyper_profiles = TVAESynthesizer(metadata,
                                               compress_dims=(256,128),
                                               decompress_dims=(128,256),
                                               embedding_dim = 64,
                                               enforce_min_max_values = False,
                                               epochs=3000)
    sdv_TVAE_hyper_profiles.add_constraints(constraints = constraint_list)

    sdv_TVAE_hyper_profiles.fit(train_data)
    sdv_TVAE_hyper_profiles = sdv_TVAE_hyper_profiles.sample(n_synth)

    return sdv_TVAE_hyper_profiles



In [25]:
for i, profile in enumerate(profiles_train_hyper_nonans_list):

    fold_synthetic = generate_TVAE_hyper_profiles(profile.drop(['ID'], axis=1), 1000)
    fold_synthetic.to_csv(f"synthetic_runner_folds/synthetic_profile_{i}.csv")


/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sdv/single_table/base.py:86: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Sampling rows: 100%|██████████| 1000/1000 [00:00<00:00, 4356.40it/s]
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sdv/single_table/base.py:86: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Sampling rows: 100%|██████████| 1000/1000 [00:00<00:00, 7599.77it/s]
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sdv/single_table/base.py:86: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Sampling rows: 100%|██████████| 1000/1000 [00:00<00:00, 7594.72it/s]
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sdv/single_table/base

In [16]:
synthetic_profiles_list = []
for i, profiles in enumerate(profiles_train_list):
    syn_profile = pd.read_csv(f"synthetic_runner_folds/synthetic_profile_{i}.csv", index_col=0)
    synthetic_profiles_list.append(syn_profile)

In [17]:
# Get the differences between each measurement point

def create_profiles_diff(profiles_to_diff):
    diff_1 = []
    diff_2 = []
    diff_3 = []
    id_list = []
    for i in np.unique(profiles_to_diff['ID']):
        
        diffs = profiles_to_diff[profiles_to_diff['ID']==i].diff()#.drop(['ID'], inplace = True)
        
        diffs_change = diffs.iloc[:,9:]
        diffs = diffs.iloc[:,1:]

        # print(diffs)
        id_list.append(i)
        diff_1.append(diffs.iloc[1])
        diff_2.append(diffs.iloc[2])
        diff_3.append(diffs.iloc[3])
    # Connect the differences to the runner profiles
    
    id_df = pd.DataFrame(id_list, columns=['ID'])
    diff_1_id = pd.concat([id_df,pd.DataFrame(diff_1).reset_index(drop=True)],axis=1)
    diff_2_id = pd.concat([id_df,pd.DataFrame(diff_2).reset_index(drop=True)],axis=1)
    diff_3_id = pd.concat([id_df,pd.DataFrame(diff_3).reset_index(drop=True)],axis=1)
    
    fold_len = len(np.unique(profiles_to_diff['ID']))

    profiles_for_diff1 = profiles_to_diff.loc[:fold_len-1]
    profiles_for_diff2 = profiles_to_diff.loc[fold_len:2*fold_len-1]
    profiles_for_diff3 = profiles_to_diff.loc[2*fold_len:3*fold_len-1]

    profiles_for_diff = pd.concat([profiles_for_diff1, profiles_for_diff2, profiles_for_diff3], axis=0).reset_index(drop=True)

    
    

    diff = pd.concat([diff_1_id, diff_2_id, diff_3_id], axis=0)
    first_step = profiles_to_diff.iloc[:diff.shape[0],:]
    second_step = pd.DataFrame(first_step.iloc[:,1:].to_numpy() + diff.iloc[:,1:].to_numpy(), columns=baseline_columns + second_col_names[:-7])
    profile_and_secstep = pd.concat([first_step,second_step.iloc[:,8:]], axis=1)

    return profiles_for_diff, diff, profile_and_secstep





In [18]:
def divide2profileandtrajectory(df):
    profmeas12_list = []
    
    for i in np.unique(df['ID']):
        profiles_i = df[df['ID']==i]
        loc_weight1 = profiles_i.columns.get_loc('Weight 1')
        baselines = profiles_i.iloc[:-1,:loc_weight1].reset_index(drop=True)
        measurements = profiles_i.iloc[:,loc_weight1:]
        meas_1 = measurements.iloc[:-1].reset_index(drop=True)
        meas_2 = measurements.iloc[1:].set_axis(second_col_names[:-7],axis=1).reset_index(drop=True)
        profmeas12 = pd.concat([baselines,meas_1,meas_2],axis=1)
        
        profmeas12_list.append(profmeas12.iloc[0].reset_index(drop=True))
        profmeas12_list.append(profmeas12.iloc[1].reset_index(drop=True))
        profmeas12_list.append(profmeas12.iloc[2].reset_index(drop=True))
    profile_cols = profiles_i.columns
    meas_12_cols = profmeas12.columns
    return pd.DataFrame(profmeas12_list).set_axis(meas_12_cols,axis=1).reset_index(drop=True)

In [19]:
profMeas12_train_nans_list = []
profMeas12_test_nans_list = []

for fold, (train_index, test_index) in enumerate(kf.split(kihu_aineisto)):
    profMeas12_train = divide2profileandtrajectory(profiles_train_list[fold].isna())
    profMeas12_test = divide2profileandtrajectory(profiles_test_list[fold].isna())

    profMeas12_train_nans_list.append(profMeas12_train)
    profMeas12_test_nans_list.append(profMeas12_test)

In [20]:
asd = divide2profileandtrajectory(profiles_train_list[0].isna())

In [21]:
id_list_2manynans_train = []
for df in profiles_train_list:
    mask = np.sum(divide2profileandtrajectory(df).isna(),axis=1)<divide2profileandtrajectory(df).shape[1]/2
    
    id_list_2manynans_train.append(divide2profileandtrajectory(df)[mask])
    print(divide2profileandtrajectory(df)[mask].shape)

(269, 59)
(271, 59)
(268, 59)
(269, 59)
(265, 59)
(273, 59)
(267, 59)
(271, 59)
(269, 59)
(269, 59)


In [22]:
id_list_2manynans_test = []
for df in profiles_test_list:
    mask = np.sum(divide2profileandtrajectory(df).isna(),axis=1)<divide2profileandtrajectory(df).shape[1]/2
    
    id_list_2manynans_test.append(divide2profileandtrajectory(df)[mask])
    print(divide2profileandtrajectory(df)[mask].shape)

(30, 59)
(28, 59)
(31, 59)
(30, 59)
(34, 59)
(26, 59)
(32, 59)
(28, 59)
(30, 59)
(30, 59)


In [23]:
profMeas12_train_list = []
profMeas12_test_list = []

for fold, (train_index, test_index) in enumerate(kf.split(kihu_aineisto)):
    profMeas12_train = divide2profileandtrajectory(profiles_train_list[fold])
    profMeas12_test = divide2profileandtrajectory(profiles_test_list[fold])

    profMeas12_train_list.append(profMeas12_train)
    profMeas12_test_list.append(profMeas12_test)

    

In [37]:
profMeas12_train_hyper_list =[]
profMeas12_test_hyper_list =[]

for fold, (train_index, test_index) in enumerate(kf.split(kihu_aineisto)):
    id_column_train = profMeas12_train_list[fold]['ID']
    id_column_test = profMeas12_test_list[fold]['ID']
    
    imputer_for_pandd = imputers.get("hyperimpute")
    profMeas12_hyper_train = imputer_for_pandd.fit_transform(profMeas12_train_list[fold].iloc[:,1:]) #drop ID from imputation
    profMeas12_hyper_test = imputer_for_pandd.transform(profMeas12_test_list[fold].iloc[:,1:])

    profMeas12_train_hyper_list.append(pd.concat([id_column_train, profMeas12_hyper_train],axis=1)) # Add it back after
    profMeas12_test_hyper_list.append(pd.concat([id_column_test,profMeas12_hyper_test],axis=1))



/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/pekkamela/wdl/wdl_workfolder/.venv/lib/python3.10

In [38]:

def diff_from_pnd(pnd):
    meas1 = pnd.iloc[:,pnd.columns.get_loc('Weight 1'):pnd.columns.get_loc('Weight 2')]
    meas2 = pnd.iloc[:,pnd.columns.get_loc('Weight 2'):]
    diff = meas2.to_numpy() - meas1.to_numpy()
    diff = pd.DataFrame(diff, columns=pnd.iloc[:,pnd.columns.get_loc('Weight 2'):].columns)
    profile = pnd.iloc[:,:pnd.columns.get_loc('Weight 2')]
    return profile, diff


In [39]:
def predict_next_step_nocond(last_step, new_col_names, predictor):
    predictions = predictor.predict(last_step.to_numpy())

    # zeros for non-changing variable deltas
    baseline_pad = np.zeros((1000,8))

    # Difference dataframe predicted for the synthesized runner profiles
    # TODO: is dataframing needed at this point?
    synth_diff = pd.DataFrame(np.concatenate( (baseline_pad, predictions), axis=1),
                              columns=new_col_names).round(rounding_scheme)

    synth_next_step = []

    for i, row in enumerate(last_step.values):
        synth_next_step.append(row+synth_diff.iloc[i].to_numpy())
    
    synth_next_step_df = pd.DataFrame(synth_next_step,columns=new_col_names)

    synth_next_step_df_rounded = synth_next_step_df.round(rounding_scheme)

    return synth_next_step_df_rounded, synth_diff

In [40]:
def zero_constraint_mse_loss(y_true, y_pred):
    error = y_true-y_pred    
    #square of the error
    sqr_error = tensorflow.square(error)
    #mean of the square of the error
    mean_sqr_error = tensorflow.reduce_mean(sqr_error)

    is_negative = (y_pred < 0)
    zero_constraint_loss = tensorflow.abs(error)*50


    return tensorflow.where(is_negative, zero_constraint_loss, mean_sqr_error)


In [24]:
profMeas12_train_nonans_list = []

for i, profmeas12 in enumerate(profMeas12_train_list):
    profmeas12_nonans = rm_half_nan(profmeas12, profmeas12)
    profMeas12_train_nonans_list.append(profmeas12_nonans)


profMeas12_test_nonans_list = []

for i, profmeas12 in enumerate(profMeas12_test_list):
    profmeas12_nonans = rm_half_nan(profmeas12, profmeas12)
    profMeas12_test_nonans_list.append(profmeas12_nonans)


In [27]:
for i, profmeas12 in enumerate(profMeas12_train_nonans_list):
    profmeas12.to_csv(f"./final_sets/real_train/profiles_and_diff_nan_{i}.csv")
    profMeas12_test_nonans_list[i].to_csv(f"./final_sets/real_test/profiles_and_diff_nan_{i}.csv")



In [41]:
profMeas12_train_hyper_nonans_list = []

for i, profmeas12 in enumerate(profMeas12_train_hyper_list):
    profmeas12_nonans = rm_half_nan(profmeas12, profMeas12_train_list[i])
    profMeas12_train_hyper_nonans_list.append(profmeas12_nonans)


profMeas12_test_hyper_nonans_list = []

for i, profmeas12 in enumerate(profMeas12_test_hyper_list):
    profmeas12_nonans = rm_half_nan(profmeas12, profMeas12_test_list[i])
    profMeas12_test_hyper_nonans_list.append(profmeas12_nonans)


In [42]:
for df in profMeas12_train_hyper_nonans_list:
    print(df.shape)
for df in profMeas12_test_hyper_nonans_list:
    print(df.shape)

(269, 59)
(271, 59)
(268, 59)
(269, 59)
(265, 59)
(273, 59)
(267, 59)
(271, 59)
(269, 59)
(269, 59)
(30, 59)
(28, 59)
(31, 59)
(30, 59)
(34, 59)
(26, 59)
(32, 59)
(28, 59)
(30, 59)
(30, 59)


In [60]:
from tensorflow.keras.callbacks import EarlyStopping

In [64]:
diff_predictor_list = []
synthetic_profiles_12_list = []
synth_diff12_list = []
#TODO testisetistä testaus
for i in range(0,10):
    x_train_df, y_train_df = diff_from_pnd(profMeas12_train_hyper_nonans_list[i])
    x_test_df, y_test_df = diff_from_pnd(profMeas12_test_hyper_nonans_list[i])
    x_train, y_train = x_train_df.to_numpy(), y_train_df.to_numpy()
    x_test, y_test = x_test_df.to_numpy(), y_test_df.to_numpy()
    # No need for ID column
    x_train = x_train[:, 1:]
    x_test = x_test[:, 1:]

    model_input = tf.keras.Input(shape=x_train[1,:].shape)
    x = tf.keras.layers.Normalization()(model_input)
    x = tf.keras.layers.Dense(128, activation='selu')(x)
    # x = tf.keras.layers.Dense(128, activation='selu')(model_input)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation='selu')(x)
    x = tf.keras.layers.Dense(64, activation='selu')(x)
    x = tf.keras.layers.Dense(64, activation='selu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation='selu')(x)
    model_output = tf.keras.layers.Dense(y_train.shape[1], activation='linear')(x)
    diff_predictor = tf.keras.Model(model_input,model_output)
    # diff_predictor.summary()

    diff_predictor.compile(optimizer='adam', loss=losses.MeanSquaredError(),metrics=[
        metrics.MeanSquaredError(name='my_mse'),
    ])

    early_stopping = EarlyStopping(monitor='val_loss', patience=1000, restore_best_weights=True)

    diff_predictor.fit(x_train,y_train, epochs=1000, verbose=False, validation_data=(x_test, y_test),
    callbacks=[early_stopping])

    
    step_2_df, synth_diff12 = predict_next_step_nocond(synthetic_profiles_list[i], cols_baseline_measure_2[:-7], diff_predictor)
    # (loss,mse_test) = diff_predictor.evaluate(x_test, y_test, verbose=1)
    diff_predictor_list.append(diff_predictor)
    step_12_df = pd.concat([synthetic_profiles_list[i].round(rounding_scheme), step_2_df.iloc[:,8:]],axis=1)
    synthetic_profiles_12_list.append(step_12_df)
    synth_diff12_list.append(synth_diff12)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [63]:
for i in range(0,10):
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(synthetic_profiles_12_list[i])

    test_original = profMeas12_train_hyper_nonans_list[i].drop(['ID'],axis=1)
    test_synthetic = synthetic_profiles_12_list[i]

    hyperimpute_qr = evaluate_quality(
        real_data=test_original,
        synthetic_data=test_synthetic,
        metadata=metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 58/58 [00:00<00:00, 1416.74it/s]|
Column Shapes Score: 85.97%

(2/2) Evaluating Column Pair Trends: |██████████| 1653/1653 [00:07<00:00, 215.32it/s]|
Column Pair Trends Score: 82.27%

Overall Score (Average): 84.12%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 58/58 [00:00<00:00, 1536.81it/s]|
Column Shapes Score: 89.32%

(2/2) Evaluating Column Pair Trends: |██████████| 1653/1653 [00:07<00:00, 223.44it/s]|
Column Pair Trends Score: 83.16%

Overall Score (Average): 86.24%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 58/58 [00:00<00:00, 1344.81it/s]|
Column Shapes Score: 83.33%

(2/2) Evaluating Column Pair Trends: |██████████| 1653/1653 [00:07<00:00, 216.90it/s]|
Column Pair Trends Score: 81.33%

Overall Score (Average): 82.33%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 58/58 [00:00<00:00, 1513.04it/s]|
Column Shapes Score: 88.39%

(2/2) Evaluati

In [55]:
synthetic_profiles_12_cleaned_list = []
for df in synthetic_profiles_12_list:
    synthetic_profiles_12_cleaned_list.append((df>0)*df)

In [56]:
for i in range(0,10):
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(synthetic_profiles_12_list[i])

    test_original = profMeas12_train_hyper_nonans_list[i].drop(['ID'],axis=1)
    test_synthetic = synthetic_profiles_12_cleaned_list[i]

    hyperimpute_qr = evaluate_quality(
        real_data=test_original,
        synthetic_data=test_synthetic,
        metadata=metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 58/58 [00:00<00:00, 1585.90it/s]|
Column Shapes Score: 88.86%

(2/2) Evaluating Column Pair Trends: |██████████| 1653/1653 [00:07<00:00, 209.79it/s]|
Column Pair Trends Score: 83.32%

Overall Score (Average): 86.09%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 58/58 [00:00<00:00, 1456.02it/s]|
Column Shapes Score: 89.63%

(2/2) Evaluating Column Pair Trends: |██████████| 1653/1653 [00:07<00:00, 214.18it/s]|
Column Pair Trends Score: 83.68%

Overall Score (Average): 86.66%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 58/58 [00:00<00:00, 1565.69it/s]|
Column Shapes Score: 88.05%

(2/2) Evaluating Column Pair Trends: |██████████| 1653/1653 [00:07<00:00, 209.37it/s]|
Column Pair Trends Score: 83.67%

Overall Score (Average): 85.86%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 58/58 [00:00<00:00, 1603.30it/s]|
Column Shapes Score: 89.4%

(2/2) Evaluatin

In [57]:
for i in range(0,10):
    synthetic_profiles_12_cleaned_list[i].to_csv(f"./final_sets/synth/synthetic_profiles_12_{i}.csv")

In [58]:
for i in range(0,10):
    profMeas12_train_hyper_nonans_list[i].to_csv(f"./final_sets/real_train/profiles_and_diff_{i}.csv")
    profMeas12_test_hyper_nonans_list[i].to_csv(f"./final_sets/real_test/profiles_and_diff_{i}.csv")